In [1]:
########################25#######################################
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from lightgbm import LGBMRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [2]:
plt.style.use('fivethirtyeight')
df = pd.read_csv('train.csv')
# data= pd.merge(data,feature,on='date',how='left')
df['date'] = pd.to_datetime(df['date'],format='%d/%m/%Y %H:%M')



In [3]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X
X_train,y=create_features(df, label='speed')
X_train = X_train.to_numpy()
y = y.to_numpy()



<ipython-input-3-8e5564f76355>:12: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df['date'].dt.weekofyear


In [4]:
# 归一化X，归一化y
scaler_X = preprocessing.StandardScaler()
scaler_X.fit(X_train) 
X_train = scaler_X.transform(X_train)

scaler_y = preprocessing.MinMaxScaler()
scaler_y.fit(y.reshape(-1,1))
y = scaler_y.transform(y.reshape(-1,1))

reg = CatBoostRegressor(depth=8, n_estimators=5000,eval_metric='MAE',l2_leaf_reg=3,verbose=0)
reg.fit(X_train,y)
C = pd.read_csv('test.csv')
C['date'] = pd.to_datetime(C['date'],format='%d/%m/%Y %H:%M')



In [5]:
def create_features(df):
    """
    Creates time series features from datetime index
    """
    
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek

    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X1 = df[['hour','dayofweek','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    return X1


In [6]:
X1=create_features(C)
X1=scaler_X.transform(X1)
Y=reg.predict(X1)
Y=scaler_y.inverse_transform(Y.reshape(-1,1))
Y=pd.DataFrame(Y)
Y['id']=C['id']
Y['speed']=Y[0]
del Y[0]
Y.to_csv('catboostbest.csv',index=False,header=1)

<ipython-input-5-31b7ab8ccf83>:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df['date'].dt.weekofyear
